In [ ]:
#====================================================================================
#LIBRARIES
#====================================================================================
import json
import calendar
import pandas as pd
import geopandas as gpd
from geojson_rewind import rewind
import numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'chrome'
import plotly.graph_objects as go


#specify current year/month
curr_yr = 2023
curr_mo = 9


#====================================================================================
#READ IN DATA
#====================================================================================

#top brands
top_brands_df = pd.read_csv('app_data/top_brands.csv')

#monthly time series
mnly_ts_df = pd.read_csv('app_data/mnly_ts_data.csv')

#MAP DATA
#geojson data
geojson_path = 'app_data/geo_data.geojson'
with open(geojson_path,'r') as f:
    counties = json.load(f)
#county ids
county_id_map = {}
for i,feature in enumerate(counties['features']):
    id_num = feature['properties']['OBJECTID']
    county_name = feature['properties']['JURISDICT_LABEL_NM']
    feature['id'] = id_num
    county_id_map[county_name] = id_num
#rewind geojson
counties = rewind(counties,rfc7946 = False)

#map df
map_df = pd.read_csv('app_data/map_data.csv')


#===========================================================================================================
#helper functions
#===========================================================================================================
def get_county_str(x):
        if 'Unknown' in x:
            return 'Counties'
        if 'Statewide' in x:
            return ''
        return 'County'
    
special_fmt_brands = {'gmc':'GMC',
                      'bmw':'BMW',
                      'mercedesbenz':'Mercedes-Benz'}
def get_formatted_brand_name(x):
    if x.lower() in special_fmt_brands:
        return special_fmt_brands[x.lower()]
    else:
        return x.title()
    
def get_map_title(brand, fuel, metric):
    title_main = f'{curr_yr} Registrations by County'

    title_brand = get_formatted_brand_name(brand)
    if brand == 'allbrands':
        title_brand = 'All Brands'
    
    title_fuel = fuel.title()
    if fuel == 'allfueltypes':
        title_fuel = 'All Fuel Types'

    if 'count' in metric.lower():
        title_metric = 'Volume'
        if (title_fuel == 'All Fuel Types'):
            if (title_brand == 'All Brands'):
                title_sub = f'({title_metric} for {title_brand}, {title_fuel})'
            else:
                title_sub = f'({title_metric} for {title_brand}, {title_fuel})'
        elif (title_brand == 'All Brands'):
            title_sub = f'({title_metric} for {title_brand}, {title_fuel} Vehicles)'
        else:
            title_sub = f"({title_metric} for {title_brand} {title_fuel} Vehicles)"
               
    elif 'percap' in metric.lower():
        title_metric = 'Volume per Capita'
        if (title_fuel == 'All Fuel Types'):
            if (title_brand == 'All Brands'):
                title_sub = f'({title_metric} for {title_brand}, {title_fuel})'
            else:
                title_sub = f'({title_metric} for {title_brand}, {title_fuel})'
        elif (title_brand == 'All Brands'):
            title_sub = f'({title_metric} for {title_brand}, {title_fuel} Vehicles)'
        else:
            title_sub = f"({title_metric} for {title_brand} {title_fuel} Vehicles)"
        
    elif 'shareft' in metric.lower():
        title_metric = 'Fuel Share'
        if (title_fuel == 'All Fuel Types'):
            if (title_brand == 'All Brands'):
                title_sub = f"({title_fuel}' {title_metric} for {title_brand})"
            else:
                title_sub = f"({title_fuel}' {title_metric} for {title_brand} Vehicles)"
        elif (title_brand == 'All Brands'):
            title_sub = f'({title_fuel} {title_metric} for All Brands)'
        else:
            title_sub = f"({title_fuel} {title_metric} for {title_brand} Vehicles)"
            
    elif 'sharebr' in metric.lower():
        title_metric = 'Brand Share'
        if (title_fuel == 'All Fuel Types'):
            if (title_brand == 'All Brands'):
                title_sub = f"({title_brand}' {title_metric} for {title_fuel})"
            else:
                title_sub = f"({title_brand} {title_metric} for {title_fuel})"
        elif (title_brand == 'All Brands'):
            title_sub = f"({title_brand}' {title_metric} for {title_fuel} Vehicles)"
        else:
            title_sub = f"({title_brand} {title_metric} for {title_fuel} Vehicles)"
    
    title = title_main + '<br>' + title_sub

    return title



#styles
dupdate_style = {'width':'100%', 
                 'height':'30px',
                 'display':'inline-block',
                 'margin-right':'1%',
                 'margin-left':'1%',
                 'margin-top':'0px',
                 'margin-bottom':'4px',
                 'justify-content':'center',
                 'justify':'center',
                 'align':'center',
                 'align-items':'center',
                 'background-color':'rgb(255,255,255)',
                 'border-radius': '2px',
                 'text-align':'center', 
                 'padding':'1px'}

dh_style = {'width':'100%', 
            'height':'120px',
            'display':'inline-block',
            'margin-right':'1%',
            'margin-left':'1%',
            'margin-top':'0px',
            'margin-bottom':'10px',
            'justify-content':'center',
            'align':'center',
            'align-items':'center',
            'background-color':'rgba(255,255,255,0)',
            'border-radius': '5px',
            'text-align':'center',
            'padding':'1px'}

fullrow_div_style = {'width':'100%', 
                     'height':'350px',
                     'margin-right':'1%',
                     'margin-left':'1%',
                     'display':'flex',
                     'border': '1px solid gray',  
                     'border-radius': '5px',       
                     'box-shadow': '1px 1px 1px 0px #888888',  
                     'padding': '2px',            
                     'background-color': 'white'}

cardrow_div_style = {'width':'100%', 
                     'height':'250px',
                     'margin-right':'1%',
                     'margin-left':'1%',
                     'display':'flex',
                     'border': '1px solid gray',  
                     'border-top-left-radius': '5px', 
                     'border-top-right-radius': '5px',   
                     'border-bottom-left-radius': '5px',   
                     'border-bottom-right-radius': '5px',   
                     'box-shadow': '1px 1px 1px 0px #888888',  
                     'padding': '2px',            
                     'background-color': 'white'}

card_div_style = {'width':'100%', 
                  'height':'250px',
                  'display':'flex',
                  'padding': '0px',  
                  'margin':'0px',
                  'align':'center',
                  'background-color': 'white'}

footnotes_div_style = {'width':'100%',
                      'height':'165px',
                      'margin-right':'1%',
                      'margin-left':'1%',
                      'margin-top':'15px',
                      'display':'flex',
                      'font-family': 'Helvetica, Arial, sans-serif',
                      'align':'left',
                      'font-size':'10px',
                      'justify-content':'left',
                      'border': '1px solid gray',
                      'border-radius': '5px',
                      'padding-left':'2px',
                      'padding-right':'2px',
                      'font-style':'italic',
                      'color':'rgb(50,50,50)'}

card_fig_layout = {'margin':{'l':0, 'r':0, 't':70, 'b':0}, 
                   'width':300, 'height':250}

fullrow_g_style = {'width':'95%',
                   'height':'100%', 
                   'margin':'auto',
                   'padding':'0px',
                   'align':'left',
                   'display':'inline-block'}
double_g_style = {'width':'95%',
                   'height':'100%',
                   'padding':'0px',
                   'align':'left',
                   'display':'block',
                   'margin':{'l':0, 'r':0, 't':0, 'b':0},
                   'width':350, 'height':350} #todo: delete

#other vars
color_map = {'Gasoline':'blue',
             'Hybrid':'purple',
             'Electric':'red',
             'Diesel':'black',
             'Other':'lightgray'}

map_brand_options_lst = ['All Brands'] + sorted(set([i.split('_')[1] for i in map_df.columns if (len(i.split('_'))>1) and (i.split('_')[1]!='allbrands')]) - {'allbrands'})
map_brand_options = [{'label':get_formatted_brand_name(i.lower()),'value':i.lower().replace(' ','')} for i in map_brand_options_lst]

dropdown_style = {'width': '150px',
                  'height':'30px',
                  'display': 'block', 
                  'padding':'2px',
                  'align':'center',
                  'align-items':'center',
                  'font-size':'14px',
                  'font-family': 'Helvetica, Arial, sans-serif'}

dropdown_panel_style = {'background-color':'rgb(240,240,240)',
                        'width':'160px',
                        'align-items':'center',
                        'justify-items':'center',
                        'display':'inline-block',
                        'white-space':'normal',
                        'border-right': '1px solid gray',
                        'font-size':'14px',
                        'font-family': 'Helvetica, Arial, sans-serif'}

layout_style = {'width':'970px',
                'display':'inline-block', 
                'align-items':'center', 
                'justify-content':'center',
                'align':'center'}

white_to_blue_cs = [f"rgba(0,0,255,{i/100})" for i in range(1,101)]

plot_bgcolor = 'rgb(245,245,245)'


#card figs

#c1 fig
total_regs_card_df = pd.read_csv('app_data/total_regs_card.csv')
curr_yr_tot = total_regs_card_df.loc[total_regs_card_df['year']==curr_yr,'count'].iloc[0]
prev_yr_tot = total_regs_card_df.loc[total_regs_card_df['year']==curr_yr-1,'count'].iloc[0]
c1fig = go.Figure()
c1fig.add_trace(go.Indicator(
    mode = 'number+delta', value = curr_yr_tot,
    title = {'text': f"<span 'style='font-size:0.8em'>{curr_yr} Total<br>Registrations<br></span><span style='font-size:0.7em;font-weight:bold;color:rbg(10,10,10)'> </span>"},
    delta = {'reference': prev_yr_tot, 'relative': True, 'valueformat': '.2%',
             'suffix':' YOY','increasing': {'color': 'blue'}, 'decreasing': {'color': 'red'}}))
c1fig.update_layout(**card_fig_layout)

#c2 fig
top_brand_card_df = pd.read_csv('app_data/top_brand_card.csv')
top_brand = top_brand_card_df.loc[total_regs_card_df['year']==curr_yr,'brand'].iloc[0]
curr_yr_tot = top_brand_card_df.loc[total_regs_card_df['year']==curr_yr,'count'].iloc[0]
prev_yr_tot = top_brand_card_df.loc[total_regs_card_df['year']==curr_yr-1,'count'].iloc[0]
c2fig = go.Figure()
c2fig.add_trace(go.Indicator(
    mode = 'number+delta', value = curr_yr_tot,
    title = {"text": f"<span 'style='font-size:0.8em'>{curr_yr} Top Brand<br>by Registrations<br></span><span style='font-size:1em;font-weight:bold;color:rbg(10,10,10)'>{top_brand.upper()}</span>"},
    delta = {'reference': prev_yr_tot, 'relative': True, 'valueformat': '.2%',
             'suffix':' YOY','increasing': {'color': 'blue'}, 'decreasing': {'color': 'red'}}))
c2fig.update_layout(**card_fig_layout)

#c3 fig
fuel_card_df = pd.read_csv('app_data/fuel_card.csv')
assert fuel_card_df['year'].max()==curr_yr
pct_ch_fuels = ['Gasoline','Hybrid','Electric','Diesel']
mx_idx = fuel_card_df[pct_ch_fuels].pct_change().iloc[1].argmax()
greatest_fuel = fuel_card_df[pct_ch_fuels].pct_change().iloc[1].index[mx_idx]
greatest_fuel_val = fuel_card_df[pct_ch_fuels].iloc[1].values[mx_idx]
greatest_fuel_ref = fuel_card_df[pct_ch_fuels].iloc[0].values[mx_idx]
c3fig = go.Figure()
c3fig.add_trace(go.Indicator(
    mode = 'number+delta', value = greatest_fuel_val,
    title = {'text': f"<span 'style='font-size:0.8em'>{curr_yr} Fuel with Greatest YOY<br>% Increase in Registrations<br></span><span style='font-size:1em;font-weight:bold;color:rbg(10,10,10)'>{greatest_fuel.upper()}</span>"},
    delta = {'reference': greatest_fuel_ref, 'relative': True, 'valueformat': '.2%',
             'suffix':' YOY','increasing': {'color': 'blue'}, 'decreasing': {'color': 'red'}}))
c3fig.update_layout(**card_fig_layout)




#====================================================================================
#DEFINE DIVS
#====================================================================================

#data updated
update_row_div = html.Div([
                     html.P([
                         "Data updated through: ", 
                         html.Span(f"{calendar.month_name[curr_mo]} {curr_yr}", style = {'font-weight':'bold'})
                         ],
                         style = {
                             'font-family': 'Helvetica, Arial, sans-serif',
                             'align':'top',
                             'align-items':'top',
                             'font-size':'15px',
                             'justify-content':'center',
                             'justify':'center',
                             'color':'rgb(60,60,60)',
                             'margin-bottom':'2px'
                         }
                     )
                 ], style = dupdate_style)


#header
header_row_div = html.Div([
                     html.H1([
                         'Washington State',
                         html.Br(),
                         'Vehicle Registration Dashboard',
                         html.Br(),' '
                         ],
                         style = {
                             'font-family': 'Helvetica, Arial, sans-serif',
                             'align':'center',
                             'font-size':'38px',
                             'justify-content':'center'
                         }
                     )
                     ],
                     style = dh_style
                 )

#cards
cards_row_div = html.Div([
                    html.Div([
                        dcc.Graph(
                            figure = c1fig,
                            style = fullrow_g_style,
                            config = {'displayModeBar':False}
                        )], 
                        style = card_div_style
                    ),
                    html.Div([
                        dcc.Graph(
                            figure = c2fig,
                            style = fullrow_g_style,
                            config = {'displayModeBar':False}
                        )],
                        style = card_div_style
                    ),
                    html.Div([
                        dcc.Graph(
                            figure = c3fig,
                            style = fullrow_g_style,
                            config = {'displayModeBar':False}
                        )
                    ],
                    style = card_div_style
                    )
                ], 
                style = cardrow_div_style
                )


#map
map_row_div = html.Div([
                  html.Div([
                      """Select Brand:""",
                      dcc.Dropdown(
                          id = 'map-dd-1',
                          options = map_brand_options,
                          value = 'allbrands',
                          clearable = False,
                          searchable = False,
                          style = dropdown_style
                      ),
                      html.Br(),
                      """Select Fuel:""",
                      dcc.Dropdown(
                          id = 'map-dd-2',
                          options = [
                              {'label':'All Fuel Types','value':'allfueltypes'},
                              {'label':'Gasoline',      'value':'gasoline'},
                              {'label':'Hybrid',        'value':'hybrid'},
                              {'label':'Electric',      'value':'electric'},
                              {'label':'Diesel',        'value':'diesel'},
                          ],
                          value = 'allfueltypes',
                          clearable = False,
                          searchable = False,
                          style = dropdown_style
                      ),
                      html.Br(),
                      """Select Metric:""",
                      dcc.Dropdown(
                          id = 'map-dd-3',
                          options = [
                              {'label':'Volume',            'value':'count'},
                              {'label':'Volume per Capita', 'value':'percap'},
                              {'label':"Fuel Type Share",  'value':'shareft'},
                              {'label':"Brand Share",      'value':'sharebr'}
                          ],
                          value = 'count',
                          clearable = False,
                          searchable = False,
                          style = dropdown_style
                      )
                  ],
                  style = dropdown_panel_style
                  ),
                  html.Div([
                      dcc.Graph(
                          id = 'map-graph',
                          style = fullrow_g_style,
                          config = {'displayModeBar':False}
                      ),
                      ], style = {'width':'100%'}
                      )
              ],
              style = fullrow_div_style
              )


#top brands
top_brands_row_div = html.Div([
                         html.Div([
                             """ Select Year:""",
                             dcc.Dropdown(
                                 id = 'bar-dd-1',
                                 options = sorted(top_brands_df['year'].unique(), reverse = True),
                                 value = top_brands_df['year'].max(),
                                 clearable = False,
                                 searchable = False,
                                 style = dropdown_style
                             ),
                             html.Br(),
                             """ Select County:""",
                             dcc.Dropdown(
                                 id = 'bar-dd-2',
                                 options = ['Statewide'] + sorted(set(top_brands_df['county']) - set(['Statewide'])),
                                 value = 'Statewide',
                                 clearable = False,
                                 searchable = False,
                                 style = dropdown_style
                             )
                         ],
                         style = dropdown_panel_style
                         ),
                         html.Div([
                             dcc.Graph(
                                 id = 'bar-graph', 
                                 style = fullrow_g_style,
                                 config = {'displayModeBar':False}
                             )
                         ], 
                         style = {'width':'100%'})
                    ],
                    style = fullrow_div_style
                    )

#scatterplot
scatterplot_row_div = html.Div([
                          html.Div([
                              dcc.RadioItems(
                                    id = 'scatter-radio',
                                    options=[
                                        {'label': 'Brand Analysis', 'value': 'brand'},
                                        {'label': 'Fuel Analysis', 'value': 'fuel'}
                                    ],
                                    value = 'brand',
                                    labelStyle = {'display': 'block'}
                              ),
                              html.Br(),
                              html.Div(id = 'new-dd-text1'),
                              dcc.Dropdown(
                                  id = 'scatter-dd-1',
                                  clearable = False,
                                  searchable = False,
                                  style = dropdown_style
                                  ),
                              html.Br(),
                              """Select Y-Axis Metric:""",
                              dcc.Dropdown(
                                  id = 'scatter-dd-2',
                                  clearable = False,
                                  searchable = False,
                                  style = dropdown_style
                                  ),
                              html.Br(),
                              """Select X-Axis Metric:""",
                              dcc.Dropdown(
                                  id = 'scatter-dd-3',
                                  options = [
                                      {'label':'Median Income','value':'medinc'},
                                      {'label':'Population Density','value':'popdensity'}
                                  ],
                                  value = 'medinc',
                                  clearable = False,
                                  searchable = False,
                                  style = dropdown_style
                                  ),
                              html.Br(),
                              """Select Line:""",
                              dcc.Dropdown(
                                  id = 'scatter-dd-4',
                                  options = [
                                      {'label':'OLS','value':'ols'},
                                      {'label':'LOESS','value':'lowess'}
                                  ],
                                  value = 'ols',
                                  clearable = False,
                                  searchable = False,
                                  style = dropdown_style
                                  )
                              ],
                              style = dropdown_panel_style),
                          html.Div([
                              dcc.Graph(
                                  id = 'scatter-graph',
                                  style = fullrow_g_style,
                                  config = {'displayModeBar':False}
                                  ),
                              ],
                              style = {'width':'100%'})
                          ],
                          style = fullrow_div_style
                      )


#monthly time series -- brand, county, fuel type
mnlyts_row_div = html.Div([
                          html.Div([
                              """Select Brand:""",
                              dcc.Dropdown(
                                  id = 'ts-dd-1',
                                  options = ['All Brands'] + sorted(set(mnly_ts_df['brand']) - {'All Brands'}),
                                  value = 'All Brands',
                                  clearable = False,
                                  searchable = False,
                                  style = dropdown_style
                                  ),
                              html.Br(),
                              """Select County:""",
                              dcc.Dropdown(
                                  id = 'ts-dd-2',
                                  options = ['Statewide'] + sorted(set(mnly_ts_df['county']) - {'Statewide'}),
                                  value = 'Statewide',
                                  clearable = False,
                                  searchable = False,
                                  style = dropdown_style
                                  ),
                              html.Br(),
                              """Select Metric:""",
                              dcc.Dropdown(
                                  id = 'ts-dd-3',
                                  options = [
                                      {'label':'Fuel Volumes','value':'volume'},
                                      {'label':'Fuel Shares','value':'share'}
                                  ],
                                  value = 'volume',
                                  clearable = False,
                                  searchable = False,
                                  style = dropdown_style
                                  )
                              ],
                              style = dropdown_panel_style),
                          html.Div([
                              dcc.Graph(
                                  id = 'ts-graph',
                                  style = fullrow_g_style,
                                  config = {'displayModeBar':False}
                                  ),
                              ],
                              style = {'width':'100%'})
                          ],
                          style = fullrow_div_style
                      )

#footnote div
footnote_div = html.Div([
                     html.P([
                         html.Div(["""Notes:"""], style = {'font-weight':'bold', 'text-decoration':'underline'}),
                         html.Div(["""- This dashboard has no affiliation with the State of Washington."""],
                                  style = {'margin-left':'10px', 'margin-top':'0px','margin-bottom':'0px'}),
                         html.Div(["""- Data includes only original vehicle registrations; it excludes registration renewals and transfers."""],
                                  style = {'margin-left':'10px', 'margin-top':'0px','margin-bottom':'0px'}),
                         html.Div(["""- The brand options in the "Select Brand" dropdown menus are limited to the top brands by original registration volume for the most recent year in the data."""],
                                  style = {'margin-left':'10px', 'margin-top':'0px','margin-bottom':'0px'}),
                         html.Div(["""- Fuel categories are defined here as follows: 
                         Gasoline vehicles are those with a primary fuel type of "Gasoline" and a secondary fuel type of NULL, "Gasoline," "Ethanol (E85)," or "Flexible Fuel Vehicle (FFV)"; 
                         Hybrid vehicles are those with primary fuel type of "Hybrid" with any secondary fuel type, or a primary fuel type of "Gasoline" and a secondary fuel type of "Electric," or a primary fuel type of "Electric" and a secondary fuel type of "Gasoline";
                         Electric vehicles are those with a primary fuel type of "Electric" and a secondary fuel type of NULL;
                         Diesel vehicles are those with a primary fuel type of "Diesel" and a secondary fuel type of NULL;
                         the Other category consists of all vehicles that do not meet any of these definitions."""],
                                  style = {'margin-left':'10px', 'margin-top':'0px','margin-bottom':'0px'}),
                         html.Br(),
                         html.Div(["""Sources:"""], style = {'font-weight':'bold', 'text-decoration':'underline'}),
                         html.Div(["""- Vehicle registration data is from the """,html.A("""State of Washington's Open Data Portal.""", href = """https://data.wa.gov/Transportation/Vehicle-Registration-Transactions-by-Department-of/brw6-jymh""")], 
                                  style = {'margin-left':'10px', 'margin-top':'0px','margin-bottom':'0px'}),
                         html.Div(["""- Geographic data is from the """,html.A("""Washington Geospatial Open Data Portal.""", href = """https://geo.wa.gov/datasets/wadnr::wa-county-boundaries/explore""")], 
                                  style = {'margin-left':'10px', 'margin-top':'0px','margin-bottom':'0px'}),
                         html.Div(["""- County population and median income data are for 2020 and from the """,html.A("""U.S. Census Bureau.""", href = """https://data.census.gov/table/""")], 
                                  style = {'margin-left':'10px', 'margin-top':'0px','margin-bottom':'0px'}),
                     ],
                         style = {'padding-right': '7px','padding-left':'7px'}
                     )
                     ],
                     style = footnotes_div_style)







#====================================================================================
#APP & LAYOUT
#====================================================================================

app = Dash(__name__)

app.layout = html.Div(
                 children = [
                     header_row_div,
                     update_row_div,
                     cards_row_div,
                     top_brands_row_div,
                     mnlyts_row_div,
                     map_row_div,
                     scatterplot_row_div,
                     footnote_div,
                 ],
                 style = layout_style
             )




#====================================================================================
#CALLBACKS
#====================================================================================

#top brands bar graph
@app.callback(
    Output('bar-graph', 'figure'), 
    [Input('bar-dd-1', 'value'), 
     Input('bar-dd-2','value')]
)
def update_bar(val_a, val_b):
    year = val_a
    county = val_b
    df = top_brands_df.copy()
    df = df[(df['year']==year)&(df['county']==county)].sort_values('total', ascending = False, ignore_index = True)
    df['brand'] = df['brand'].apply(lambda x: get_formatted_brand_name(x)) 
    max_y = df['total'].max()

    fig = px.bar(df, 
             x = 'brand', 
             y = ['Gasoline', 'Hybrid', 'Electric', 'Diesel','Other'], 
             labels = {'brand':'Brand','variable':'Fuel', 'value':'Volume'},
             color_discrete_map = color_map
            )
    fig.add_trace(go.Scatter(x = df['brand'], 
                         y = df['total'],
                         text = df['data_label'],
                         mode = 'text',
                         textposition = 'top center',
                         textfont = {'size':12},
                         showlegend = False,
                         hoverinfo = 'skip'))
    fig.update_layout(barmode = 'stack', 
                  xaxis = {'categoryorder': 'total descending'},
                  yaxis = {'showgrid':True,'gridcolor':'white','range':(0, max_y*1.3)},
                  title_text = f'Top Brands Based on Registration Volume<br>({year}, {county})',
                  title_x = 0.5,
                  xaxis_title = '',
                  plot_bgcolor = plot_bgcolor,
                  margin = {'t':60, 'b':20, 'l':80, 'r':20})
    return fig

#time series graph
@app.callback(
    Output('ts-graph', 'figure'), 
    [Input('ts-dd-1', 'value'), 
     Input('ts-dd-2','value'), 
     Input('ts-dd-3','value')]
)
def update_ts(val_a, val_b, val_c):
    df = mnly_ts_df 
    df = df[(df['brand']==val_a)&(df['county']==val_b)].reset_index(drop = True).copy()
    val_cols = [i for i in df.columns if ('_' in i) and (val_c in i) and ('Total' not in i)]
    val_cols_renamed = [i.split('_')[0] for i in val_cols]
    val_cols_renaming_map = {i:i.split('_')[0] for i in val_cols}
    df = df[['dt','brand','county']+val_cols].rename(columns = val_cols_renaming_map)
    county_str = get_county_str(val_b)
    if val_c == 'volume':
        labels = {'dt':'Month','variable':'Fuel', 'value':val_c.title()}
        try:
            fig = px.area(df, x = 'dt', y = val_cols_renamed,
                          color_discrete_map = color_map,
                          line_shape = 'spline',markers = True,
                          labels = labels)
        except:
            fig = px.area(df, x = 'dt', y = val_cols_renamed,
                          color_discrete_map = color_map,
                          line_shape = 'spline',markers = True,
                          labels = labels)
        fig.update_layout(yaxis_title = 'Volume')
    elif val_c == 'share':
        labels = {'dt':'Month','variable':'Fuel', 'value':val_c.title() + ' (%)'}
        try:
            fig = px.area(df, x = 'dt', y = val_cols_renamed,
                          color_discrete_map = color_map,
                          line_shape = 'spline',markers = True,
                          labels = labels)
        except:
            fig = px.area(df, x = 'dt', y = val_cols_renamed,
                          line_shape = 'spline',markers = True,
                          labels = labels)
        fig.update_layout(yaxis = {'range':[0, 101]}) 
        fig.update_layout(yaxis_title = 'Share (%)')
    fig.update_traces(textposition = 'top center', opacity = 1)
    fig.update_layout(title_text = f'Monthly Registrations<br>({val_a.title()}, {val_b} {county_str})',
                      title_x = 0.5,
                      xaxis_title = '',
                      plot_bgcolor = plot_bgcolor,
                      legend_title_text = 'Fuel',
                      margin = {'t':60, 'b':20, 'l':80, 'r':20})
    return fig

#map
@app.callback(
    Output('map-graph', 'figure'), 
    [Input('map-dd-1', 'value'), 
     Input('map-dd-2', 'value'), 
     Input('map-dd-3', 'value')]
)
def update_map(val_a, val_b, val_c):
    var = f"{curr_yr}_{val_a.lower().replace(' ','')}_{val_b}_{val_c}"
    df = map_df[['id','county',var]]
    legend_label = 'Count'
    var_label = 'Volume'
    var_val_format = True
    if 'sharebr' in val_c.lower():
        legend_label = 'Share (%)'
        var_label = 'Brand Share (%)'
        var_val_format = ':.2f%'
    elif 'shareft' in val_c.lower():
        legend_label = 'Share (%)'
        var_label = 'Fuel Share (%)'
        var_val_format = ':.2f%'
    elif 'percap' in val_c.lower():
        var_val_format = ':.2f'
        var_label = 'Volume per 10k Persons'
    fig = px.choropleth(data_frame = df,
                    geojson = counties,
                    locationmode = 'geojson-id',
                    featureidkey = 'id',
                    locations = 'id',
                    color = var,
                    color_continuous_scale = white_to_blue_cs,
                    hover_data = {'id':False, 
                                  'county':True, 
                                  var:var_val_format},
                    labels = {var:var_label,
                             'county':'County'})
    title = get_map_title(val_a, val_b, val_c)
    fig.update_layout(title_text = title, title_x = 0.5, dragmode = False, 
                      margin = {'t':60, 'b':20, 'l':20, 'r':20},
                      coloraxis_colorbar = {'x':0.85, 'y':0.59, 'thickness':10, 'len': 0.86},
                      coloraxis_colorbar_title = legend_label)
    fig.update_geos(fitbounds = "locations", visible = False)
    return fig

#scatterplot
#radio button
@app.callback(
    [Output('new-dd-text1', 'children'),
     Output('scatter-dd-1', 'options'),
     Output('scatter-dd-1', 'value'),
     Output('scatter-dd-2', 'options'),
     Output('scatter-dd-2', 'value')],
    Input('scatter-radio', 'value')
)
def update_dd_text(val):
    if val == 'brand':
        txt = """Select Brand:"""
        optionsa = map_brand_options[1:]
        vala = map_brand_options[1]['value']
        optionsb = [{'label':'Brand Share','value':'sharebr'},
                    {'label':'Volume per Capita','value':'percap'}]
        valb = 'sharebr'
    elif val == 'fuel':
        txt = """Select Fuel:"""
        optionsa = [{'label':'Gasoline','value':'gasoline'},
                    {'label':'Hybrid','value':'hybrid'},
                    {'label':'Electric','value':'electric'},
                    {'label':'Diesel','value':'diesel'}]
        vala = 'gasoline'
        optionsb = [{'label':'Fuel Share','value':'shareft'},
                    {'label':'Volume per Capita','value':'percap'}]
        valb = 'shareft'
    return txt,optionsa,vala,optionsb,valb
    
#scatter graph
@app.callback(
    Output('scatter-graph', 'figure'), 
    [Input('scatter-dd-1', 'value'), 
     Input('scatter-dd-2','value'), 
     Input('scatter-dd-3','value'),
     Input('scatter-dd-4','value'),
     Input('scatter-radio','value')]
)
def update_scatter(val_a, val_b, val_c, val_d, val_e):
    metric = val_b
    v1 = val_c
    trendline = val_d
    analysis = val_e
    
    if v1 == 'medinc':
        v1_fmt = {'gen':'Median Income','spec':''}
    elif v1 == 'popdensity':
        v1_fmt = {'gen':'Population Density','spec':' (Persons per Square Mile)'}
        
    if analysis == 'fuel':
        v2 = f'{curr_yr}_allbrands_{val_a}_{metric}'
        val_a_fmt = val_a.title()
    elif analysis == 'brand':
        v2 = f'{curr_yr}_{val_a}_allfueltypes_{metric}'
        val_a_fmt = get_formatted_brand_name(val_a)
        
    if metric == 'percap':
        title = f"Relationship Between<br>{val_a_fmt} Vehicle Registrations per Capita and {v1_fmt['gen']} by County"
        labels = {v1:v1_fmt['gen']+v1_fmt['spec'],'county':'County',v2:f'{curr_yr} {val_a_fmt} Vehicle<br>Registrations Per 10k Persons'}
    elif metric in {'sharebr','shareft'}:
        title = f"Relationship Between<br>Share of {val_a_fmt} Vehicle Registrations and {v1_fmt['gen']} by County"
        labels = {v1:v1_fmt['gen']+v1_fmt['spec'],'county':'County',v2:f'{curr_yr} Share of {val_a_fmt}<br>Vehicle Registrations (%)'}
    
    df = map_df[['county',v1,v2]]
    df = df[df['county']!='Unknown or Out of State'].rename(columns = labels)
    try:
        fig = px.scatter(df, x = labels[v1], y = labels[v2],
                         color_discrete_sequence = ['purple'],
                         opacity = 0.9,
                         title = title, 
                         hover_data = {labels['county']:True, 
                                       labels[v1]:':,.2f',
                                       labels[v2]:':,.4f'},
                        trendline = trendline, trendline_color_override = 'black')
        fig.update_traces(marker = {'size':15,'line':{'width':1,'color':'white'}})
    except:
        fig = px.scatter(df, x = labels[v1], y = labels[v2],
                         color_discrete_sequence = ['purple'],
                         opacity = 0.9,
                         title = title, 
                         hover_data = {labels['county']:True, 
                                       labels[v1]:':,.2f',
                                       labels[v2]:':,.4f'},
                        trendline = trendline, trendline_color_override = 'black')
        fig.update_traces(marker = {'size':15,'line':{'width':1,'color':'white'}})
    fig.update_layout(title_x = 0.5, 
                      plot_bgcolor = plot_bgcolor,
                      margin = {'t':60, 'b':20, 'l':80, 'r':20}) 
    return fig
app.run_server(debug = True, port = 8000, use_reloader = False)

Dash is running on http://127.0.0.1:8000/

 * Serving Flask app '__main__'
 * Debug mode: on
